### Запуск QtLab

In [1]:
%matplotlib qt5

In [7]:
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import *
from qsweepy.ponyfiles import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
import pandas as pd
interactive(True)

In [9]:
#import resonator_quality_factor_fit
import imp
imp.reload(resonator_quality_factor_fit)
import qsweepy.save_pkl
imp.reload(qsweepy.save_pkl)
from qsweepy.save_pkl import *

In [4]:
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')

In [5]:
#pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
pna = RS_ZNB20('pna', address = 'TCPIP0::10.20.61.161::inst0::INSTR')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [15]:
switch_sample = {1:1}
sample_name = '1906-003-dodik'

In [16]:
db = database.MyDatabase()

sweeper = sweep_extras.Sweeper(db, sample_name=sample_name)
exdir_db_inst = exdir_db.Exdir_db(db=db, sample_name=sample_name)

# Прицеливание

In [58]:
#sample_name = '1901-006-1'.format(switch_sample[rf_switch.get_ch2_switch()])
sample_name = '1906-003-dodik'
#sample_name = '1902-164-1'
resonator_type='notch_port'
min_freq = 3.5e9
max_freq = 7.0e9
data_root = get_location()

save_path = '{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)

In [59]:
os.mkdir(save_path)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'c:\\data/2019-09-11/1906-003-dodik'

In [19]:
pna.set_measurement('S21')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [24]:
pna.set_average(1)
pna.set_averages(1)
pna.set_xlim(5.5e9, 7.0e9)
pna.set_nop(7501)
pna.set_bandwidth(1e3)
pna.set_power(-35)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



True

In [25]:
pna.set_sweep_mode("LIN")

True

In [26]:
fig, axes=plt.subplots(2,2, figsize=(8,8))
freqs=pna.get_freqpoints()
S21=pna.get_tracedata()
phase = np.unwrap(S21[1])
delay = np.gradient(phase)/(freqs[1]-freqs[0])
#delay_freqs = (np.asarray(freqs[1:])+np.asarray(freqs[:-1]))/2
delay_freqs=freqs
axes[0,0].plot(freqs,10*np.log10(S21[0]**2))
axes[0,1].plot(freqs, phase)
axes[1,0].plot(np.real(S21[0]*np.exp(1j*S21[1])),np.imag(S21[0]*np.exp(1j*S21[1])))
axes[1,1].plot(delay_freqs, delay)
#axes[1,1].set_yscale('log')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [28]:
sweeper.sweep(pna, 
              metadata={'probe_power':-35,
                        'bw':1000})

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



S-parameter
load time:  0.029997587203979492
pre_trace_time:  0.0
trace 0 time:  0.009999513626098633
trace 1 time:  0.020003080368041992
trace 2 time:  0.03999590873718262


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\memmap.py:335: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



trace 3 time:  0.04999828338623047


start: 2019-09-11 17:01:12.526238, started/done/total sweeps: 0/1/1.0, 
Measured data: 
'S-parameter': parameters: Frequency (Hz),:[5500000000.0, 7000000000.0] (7501 points) without setter
data: [-0.12818913+0.06625428j -0.12304553+0.07418719j -0.11850371+0.08142377j
 ...  0.07628302+0.0403444j   0.07882955+0.03560539j
  0.08103691+0.0303717j ]

In [39]:
threshold = 1e-6
baseline = -3.4e-7

In [40]:
from scipy.signal import detrend
from scipy.signal import find_peaks_cwt
from scipy.signal import argrelextrema
peaks_expected = 0.1
data = S21
data_points = len(data[0])
data_complex = 10**(data[0]/20)*np.exp(1j*detrend(np.unwrap(data[1])))
data_diff = delay-baseline#np.abs(delay-baseline)
data_diff = np.abs(delay-baseline)
data_diff[data_diff<threshold] = 0
#data_diff[data_diff>-threshold] = 0
plt.figure('delay')
plt.plot(freqs, data_diff)
data_diff=np.abs(data_diff)
#peaks = find_peaks_cwt(data_diff, np.linspace(1, 8, 4), noise_perc=100-(100.*peaks_expected)/data_points)
neighborhood_size = int(np.floor(5e6/(freqs[1]-freqs[0])))
peaks = argrelextrema(data_diff, np.greater, order=neighborhood_size)[0]
print ('found {0} peaks'.format(len(peaks)))
peak_freqs = freqs[peaks]
for freq_id, freq in enumerate(peak_freqs):
    print (freq, ',')

found 24 peaks
5725400000.0 ,
5756400000.0 ,
5790400000.0 ,
5832400000.0 ,
5865600000.0 ,
5942600000.0 ,
5978400000.0 ,
6014400000.0 ,
6063800000.0 ,
6101200000.0 ,
6137600000.0 ,
6185200000.0 ,
6225000000.0 ,
6260200000.0 ,
6312200000.0 ,
6353600000.0 ,
6391000000.0 ,
6446200000.0 ,
6486200000.0 ,
6527000000.0 ,
6583200000.0 ,
6621600000.0 ,
6632800000.0 ,
6661000000.0 ,


In [41]:
peak_freqs = [5725400000.0, 5756400000.0, 5790400000.0, 5832400000.0,
5865600000.0, 5942600000.0, 5978400000.0, 6014400000.0, 6063800000.0,
6101200000.0, 6137600000.0, 6185200000.0, 6225000000.0, 6260200000.0,
6312200000.0, 6353600000.0, 6391000000.0,
6446200000.0 ,
6486200000.0 ,
6527000000.0 ,
6583200000.0 ,
6621600000.0 ,
6632800000.0 ,
6661000000.0 ,]

In [42]:
#rescan peaks individually and fit via circlefit
from resonator_tools import circuit

typical_span=8e6
bw=30e3
nop = typical_span/bw
#pna.set_bandwidth(bw)
pna.set_nop(nop)
pna.set_span(typical_span)
pna.pre_sweep()
delay_estimate=72e-9
high_power_fits = []
for freq_id, freq in enumerate(peak_freqs):
    pna.set_centerfreq(freq)
    measurement = pna.measure()['S-parameter']
    points = pna.get_points()['S-parameter'][0][1]
    delay = np.gradient(measurement)
    fitter = circuit.reflection_port if resonator_type == 'reflection_port' else circuit.notch_port
    fitter = fitter(f_data = points, z_data_raw=measurement)
    fitter.autofit(electric_delay=delay_estimate)
    high_power_fits.append(fitter.fitresults)
    plt.figure(freq_id)
    fitter.plotall()
pna.post_sweep()
high_power_fits = pd.DataFrame(high_power_fits)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.

C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [55]:
high_power_fits = pd.DataFrame(high_power_fits)
resonator_freqs = high_power_fits['fr']
resonator_freq_spans = (high_power_fits['fr']/high_power_fits['Ql']*2).tolist()
resonator_freqs, resonator_freq_spans

(0     5.725194e+09
 1     5.756260e+09
 2     5.790752e+09
 3     5.832874e+09
 4     5.866818e+09
 5     5.945601e+09
 6     5.981864e+09
 7     6.014451e+09
 8     6.065009e+09
 9     6.101288e+09
 10    6.137971e+09
 11    6.186264e+09
 12    6.226741e+09
 13    6.260024e+09
 14    6.314141e+09
 15    6.355318e+09
 16    6.389953e+09
 17    6.447569e+09
 18    6.487906e+09
 19    6.525462e+09
 20    6.583983e+09
 21    6.622010e+09
 22    6.632944e+09
 23    6.658421e+09
 Name: fr, dtype: float64,
 [17349935.837119807,
  27276874.496349346,
  18138468.108225834,
  21340918.861829545,
  23193666.399987053,
  26359173.38418357,
  41578705.66662869,
  12682513.050614078,
  40502831.61442337,
  30913024.621952835,
  16207867.001412699,
  23307547.509090897,
  21696962.503895912,
  10241818.047132405,
  22790376.335844256,
  17657934.273384,
  11303579.584268747,
  19484993.146207515,
  21392416.93437092,
  11074147.491231875,
  18461642.862872873,
  23803585.562073328,
  15468738.17810

In [56]:
[i/1e9 for i in resonator_freqs]

[5.725193564403833,
 5.7562595235635055,
 5.790752107851705,
 5.832873771309309,
 5.866818356148766,
 5.945600532888928,
 5.981863752986668,
 6.014450522568143,
 6.065008540130309,
 6.101288070810482,
 6.13797060155955,
 6.186263792630969,
 6.226741452998216,
 6.260023716780431,
 6.314141234257169,
 6.355317787656609,
 6.3899528232640455,
 6.44756946674086,
 6.487905821668612,
 6.52546221537516,
 6.583983242124682,
 6.6220096355192615,
 6.632943536330569,
 6.658421202212813]

In [45]:
#resonator_freqs[1] = 6.3855e9
#resonator_freqs[2] = 6.6622e9
#resonator_freqs[4] = 7.07635e9
#resonator_freqs[8] = 7.637e9
#resonator_freqs[9] = 7.776e9

#resonator_freq_spans[1] = 1e6
#resonator_freq_spans[2] = 0.5e6
#resonator_freq_spans[4] = 2e6
#resonator_freq_spans[8] = 8e6
#resonator_freq_spans[9] = 8e6
#resonator_freq_spans[8] = 5e6
#resonator_freq_spans[9] = 5e6
#resonator_freq_spans[22] = 8.583e6
#resonator_freq_spans[22] = 10e6

In [ ]:
resonator_freqs, resonator_freq_spans

In [86]:
resonator_freqs = peak_freqs

In [87]:
resonator_ids = n

NameError: name 'resonator_ids' is not defined

# Мерить зонаторы

In [45]:
resonator_freqs = [ 7.108e9, 8.824e9, 7.302e9]

In [46]:
#resonator_freqs = [ 
#6.6532e9
#6.8037e9,    
#]
#powers = np.linspace(-10, -45, 36) # длинный скан
#powers = np.linspace(-20, -40, 41) # короткий скан

In [52]:
#powers = np.linspace(0, -30, 31) # короткий скан
powers = np.linspace(5, -45, 51) # длинный скан

In [48]:
resonator_ids = [i for i,f in enumerate(resonator_freqs)]
#resonator_ids = np.asarray([22, 23])-1

In [49]:
len(resonator_freqs)

24

In [50]:
import imp 
imp.reload(resonator_quality_factor_fit)

<module 'qsweepy.resonator_quality_factor_fit' from 'C:\\qtlab_replacement\\qsweepy\\resonator_quality_factor_fit.py'>

In [57]:
#resonator_freq_span = 5e6
#resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    if resonator_freq_span<1e6:
        resonator_freq_span = 1e6
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(1)
    
    
    pna.set_bandwidth(1)
    pna._visainstrument.write("SENS:AVER:MODE POIN")

    ref_pow = np.array(	[-70,	-20.,	30.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweeper.sweep(pna, 
                              (powers, set_power, 'Power'), 
                              measurement_type='resonator_powerscan',
                              type_revision='0',
                              metadata={'resonator_id': resonator_id,
                                        'freq_guess': resonator_freq,
                                        'span_guess': resonator_freq_span,
                                        'num_averages': pna.get_averages()})
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type, delay=delay_estimate)
    default_measurement_save_path(save_path, mkdir=True)
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
load time:  0.03999519348144531
pre_trace_time:  0.0
trace 0 time:  0.019999265670776367
trace 1 time:  0.03999757766723633
trace 2 time:  0.05999445915222168
trace 3 time:  0.07999801635742188


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\memmap.py:335: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
load time:  0.03999614715576172
pre_trace_time:  0.0
trace 0 time:  0.020005464553833008
trace 1 time:  0.03999519348144531
trace 2 time:  0.05999469757080078
trace 3 time:  0.07999205589294434
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-

C:\ProgramData\Anaconda3\lib\site-packages\resonator_tools\utilities.py:9: RuntimeWarning:

invalid value encountered in log10

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:436: RuntimeWarning:

Number of calls to function has reached maxfev = 600.

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.



S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
load time:  0.02999711036682129
pre_trace_time:  0.0
trace 0 time:  0.019998788833618164
trace 1 time:  0.04999709129333496
trace 2 time:  0.06999492645263672
trace 3 time:  0.08999419212341309
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-

S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
load time:  0.03999495506286621
pre_trace_time:  0.0
trace 0 time:  0.019997119903564453
trace 1 time:  0.03999614715576172
trace 2 time:  0.05999326705932617
trace 3 time:  0.07999253273010254
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:436: RuntimeWarning:

Number of calls to function has reached maxfev = 800.



S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
S-parameter
load time:  0.030018091201782227
pre_trace_time:  0.0
trace 0 time:  0.029993772506713867
trace 1 time:  0.04999232292175293
trace 2 time:  0.06997251510620117
trace 3 time:  0.08998918533325195


# load last measurements and refit

In [60]:
%matplotlib qt5
import resonator_tools.circuit
from qsweepy import *
import pickle
import matplotlib.pyplot as plt

In [62]:
save_path
#save_path=r'C:\Data\2019-02-20'
import pony.orm

In [82]:
db.Metadata.select(lambda m: m.name == 'resonator_id' and m.value== str(1)).order_by(lambda m: pony.orm.desc(m.id)).first()

Metadata[189]

In [90]:
str(m.data_id.filename)

'c:\\data\\2019-04-19\\000086-resonator_powerscan-1904-037'

In [99]:
save_path

'c:\\data/2019-04-19/1904-037'

In [172]:
import resonator_tools
imp.reload(resonator_quality_factor_fit) 

<module 'qsweepy.resonator_quality_factor_fit' from 'C:\\qtlab_replacement\\qsweepy\\resonator_quality_factor_fit.py'>

In [63]:
import glob
#measurement_pattern = r'C:\Data\2019-04-19\*.exdir'
#measurement_pattern = r'C:\Data\2019-02-20\*-1901-014-2 res {}\*.pkl'
for resonator_id in range(24):
    m = db.Metadata.select(lambda m: m.name == 'resonator_id' and m.value== str(resonator_id)).order_by(lambda m: pony.orm.desc(m.id)).first()
    measurement = save_exdir.load_exdir(str(m.data_id.filename))
    #measurements = glob.glob(measurement_pattern.format(resonator_id))
    #last_measurement = measurements[-1]
    #print (last_measurement)
    #with open(last_measurement, 'rb') as f:            
    #measurement = save_exdir.load_exdir(last_measurement, db)
    powers = measurement.datasets['S-parameter'].parameters[0].values
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, sweep_parameter_name='power', resonator_type=resonator_type, delay=delay_estimate)
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    #for i in [0]:#range(len(measurement.datasets['S-parameter'].parameters[0].values)):
    #    fitter = resonator_tools.circuit.notch_port(f_data = measurement.datasets['S-parameter'].parameters[1].values, z_data_raw=measurement.datasets['S-parameter'].data[i,:])
    #    fitter.autofit(electric_delay=delay_estimate)
    #    plt.figure()
    #    fitter.plotall()
            #plt.close()
            #plt.close()

AttributeError: Entity Metadata does not have attribute id: m.id

In [168]:
resonator_type

'notch_port'

In [116]:

plt.plot(measurement.datasets['S-parameter'].data[-1,:])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\numeric.py:501: ComplexWarning:

Casting complex values to real discards the imaginary part



In [133]:
for resonator_id in [6]:
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        print (last_measurement)
        with open(last_measurement, 'rb') as f:
            measurement = pickle.load(f)[1]['S-parameter']
            fitter = resonator_tools.circuit.reflection_port(f_data =measurement[1][1], z_data_raw=measurement[2][-1,:])
            plt.figure()
            #fitter.autofit()
            #fitter.plotall()
            #plt.figure()
            fitter.autofit(electric_delay=65e-9)
            fitter.plotall()
            #fitter.GUIfit()
            plt.show()

C:\Data\2019-02-03\03-02-12-1901-005-unknown res 6\1901-005-unknown res 6.pkl


# Загрузить фиты и построить графики

In [64]:
from matplotlib.backends.backend_pdf import PdfPages
fit_csvs = ['{0}/fit_total {1}.csv'.format(save_path, i) for i in range(len(resonator_freqs))]
rows = 6
columns = 1
figsize=(8.27, 11.69)
figs, axes = plt.subplots(rows,columns, figsize=figsize)
pdf = PdfPages('{0}/{1}.pdf'.format(save_path, sample_name))
Q_i_values_lp = []
Q_i_values_hp = []
Q_e_values = []
Q_l_values_lp = []
Q_l_values_hp = []
f_r_values = []
single_photon_limits = []
if resonator_type=='reflection_port':
    Qc_column = 'Qc'
    Qi_column = 'Qi'
    Qi_err_column = 'Qi_err'
    Ql_column = 'Ql'
else:
    Qc_column = 'Qc_dia_corr'
    Qi_column = 'Qi_dia_corr'
    Qi_err_column = 'Qi_dia_corr_err'
    Ql_column = 'Ql'
page_id = 0
for resonator_id, fit_csv in enumerate(fit_csvs):
    try:
        fitdata = pd.read_csv(fit_csv)
        #print (fitdata.shape, powers.shape)
        #if fitdata.shape[0]<len(powers):
        #fitdata['power'] = powers[(len(powers)-fitdata.shape[0]):]
        #fitdata['power'] = powers
        #print (fitdata, resonator_id)
        print (resonator_id, fitdata.shape)
        fitdata = fitdata[fitdata['fr']>4e9]
        fitdata = fitdata[fitdata['fr']<12e9]
        fitdata = fitdata[fitdata[Qc_column]>100]
        fitdata = fitdata[fitdata[Qc_column]<50000]
        print (resonator_id, fitdata.shape)
        #if resonator_id != 11:
        #fitdata = fitdata[fitdata[Qc_column]<800000]
        #else:
        #   fitdata = fitdata[fitdata['Qc_dia_corr']<2000000]
        if resonator_id and not resonator_id%(rows*columns):
            page_id +=1
            plt.tight_layout()
            pdf.savefig(figs)
            figs, axes = plt.subplots(rows, columns, figsize=figsize)
        if rows>1 and columns>1:
            Q_ax = axes[(resonator_id%(rows*columns))%rows, (resonator_id%(rows*columns))//rows]
        else:
            Q_ax = axes[resonator_id%(rows*columns)]
        Q_ax.set_yscale('log')
        Ql = Q_ax.errorbar(fitdata['power'], fitdata['Ql'], label='Q_l')#, yerr=fitdata['Ql_err'])
        Qc = Q_ax.errorbar(fitdata['power'], fitdata[Qc_column], label='Q_c')#, yerr=fitdata['absQc_err'])
        Qi = Q_ax.errorbar(fitdata['power'], np.abs(fitdata[Qi_column]), label='Q_i')#, yerr=fitdata['Qi_dia_corr_err'])
        f_ax = Q_ax.twinx()
        f = f_ax.errorbar(fitdata['power'], fitdata['fr']/1e6, color='orange', marker='.', label='$f$, MHz')#, yerr=fitdata['fr_err'])
        f_ax.set_ylabel('MHz')
        Q_ax.set_ylabel('$Q$')
        if not (fitdata['power'].size == 0):
            Q_i_values_lp.append(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])])
            Q_i_values_hp.append(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])])
            Q_e_values.append(np.median(fitdata[Qc_column]))
            Q_l_values_lp.append(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])])
            Q_l_values_hp.append(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])
            f_r_values.append(np.median(fitdata['fr']))
            single_photon_limits.append(fitdata['single_photon_limit'][np.argmax(fitdata['power'])])
            ann = f_ax.annotate('#{7}: $f_r$: {0:2.4} GHz, $Q_i$: {1:2.3g}/{3:2.3g}\n$Q_l$:{5:2.3g}/{6:2.3g}, $Q_c$: {4:2.3g}'.format(np.median(fitdata['fr'])/1e9, \
                            np.round(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])]), np.round(fitdata[Qi_err_column][np.argmin(fitdata['power'])]), \
                            np.round(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])]), np.median(fitdata[Qc_column]), \
                            np.round(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])]), np.round(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])]), \
                            resonator_id), \
                          (0.2, 0.1), xycoords='axes fraction', bbox={'alpha':0.5, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
        Q_ax.set_ylim([1e3, 1e6])
        plt.grid(True)
        if (resonator_id==0):
            leg = plt.legend([f, Qc, Qi, Ql], ['$f$', '$Q_c$', '$Q_i$', '$Q_l$'], fancybox=True, loc=6, framealpha=1)
            leg.get_frame().set_alpha(0.5)
            plt.title(sample_name)
    finally:
        pass
#axes[-1,-1].hist([Q_i_values_hp, Q_i_values_lp, Q_e_values], bins=10, label={'$Q_i$ high power', '$Q_i$ low power', '$Q_e$'})
#axes[-1,-1].set_xlabel('$Q_i$')
#axes[-1,-1].set_ylabel('resonators')
#axes[-1,-1].legend(loc='upper right')
plt.tight_layout()
pdf.savefig(figs)
pdf.close()
summary_table = pd.DataFrame({'fr':f_r_values, 'Ql_lp':Q_l_values_lp, 'Ql_hp':Q_l_values_hp, 'Qe': Q_e_values, 'Qi_lp':Q_i_values_lp, 'Qi_hp':Q_i_values_hp, 'single_photon_limits':single_photon_limits})
summary_table.to_csv('{0}/{1}.csv'.format(save_path, sample_name))

0 (51, 18)
0 (51, 18)
1 (51, 18)
1 (51, 18)
2 (51, 18)
2 (44, 18)
3 (51, 18)
3 (51, 18)
4 (51, 18)
4 (51, 18)
5 (51, 18)
5 (51, 18)
6 (51, 18)
6 (44, 18)


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning:

'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning:

'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.



7 (51, 18)
7 (51, 18)
8 (51, 18)
8 (6, 18)
9 (51, 18)
9 (51, 18)
10 (51, 18)
10 (51, 18)
11 (51, 18)
11 (51, 18)
12 (51, 18)
12 (51, 18)
13 (51, 18)
13 (51, 18)
14 (51, 18)
14 (51, 18)
15 (51, 18)
15 (51, 18)
16 (51, 18)
16 (51, 18)
17 (51, 18)
17 (51, 18)
18 (51, 18)
18 (51, 18)
19 (51, 18)
19 (51, 18)
20 (51, 18)
20 (51, 18)
21 (51, 18)
21 (50, 18)
22 (51, 18)
22 (38, 18)
23 (51, 18)
23 (51, 18)


In [65]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
(summary_table_new['Ql_lp']).round(decimals=0).astype(int)

0     76861
1      2786
2       223
3     44796
4     45519
5      3017
6       217
7       576
8       231
9      1033
10     1376
11     2263
12      429
13      177
14      648
15      358
16      926
17      233
18      456
19     1027
20      200
21    34813
22       96
23      456
Name: Ql_lp, dtype: int32

In [68]:
summary_table_new['fr']

0     5.725176e+09
1     5.756247e+09
2     5.788668e+09
3     5.832072e+09
4     5.864396e+09
5     5.941692e+09
6     5.972012e+09
7     6.012716e+09
8     6.062223e+09
9     6.098630e+09
10    6.135798e+09
11    6.182571e+09
12    6.223164e+09
13    6.260159e+09
14    6.309298e+09
15    6.350224e+09
16    6.390019e+09
17    6.443479e+09
18    6.484525e+09
19    6.526250e+09
20    6.580107e+09
21    6.629589e+09
22    6.627717e+09
23    6.663826e+09
Name: fr, dtype: float64

In [66]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
summary_table_new

,fr,Ql_lp,Ql_hp,Qe,Qi_lp,Qi_hp,single_photon_limits,k
0,5.725176e+09,76860.552925,726.313288,579.233678,50212.755254,2817.535834,-134.277764,9.884053e+06
1,5.756247e+09,2786.029083,958.898025,404.565859,1388.300116,690.932419,-138.221212,1.422821e+07
2,5.788668e+09,223.127342,450.364180,394.603965,8388.608959,3253.969379,-131.673296,1.466956e+07
3,5.832072e+09,44796.245327,1573.586288,772.508631,20181.634176,1487.913268,-139.612255,7.549523e+06
4,5.864396e+09,45518.609089,1017.186976,532.933403,14820.032407,1064.329670,-137.448831,1.100399e+07
5,5.941692e+09,3017.030500,923.701397,494.823234,1637.721476,1046.823568,-136.750452,1.200771e+07
6,5.972012e+09,216.782869,227.636143,114.896662,187.033214,232.252689,-130.843186,5.197725e+07
7,6.012716e+09,575.607103,648.750087,658.156433,16882.484071,37505.471214,-132.289818,9.135695e+06
8,6.062223e+09,230.642797,234.670497,108.436022,187.449414,223.573455,-130.998512,5.590599e+07
9,6.098630e+09,1033.475595,712.751437,326.595181,658.992421,591.558881,-136.085505,1.867336e+07


In [178]:
figs, ax = plt.subplots(1,4, figsize=(16, 5))
c = 'fr'
ax[0].plot(summary_table_new[c], marker='o')
ax[0].set_xlabel('resonator id\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[0].set_ylabel('$f_r$')
ax[0].grid(True, which='both')
c = 'Qe'
ax[1].boxplot(summary_table_new['Qe'])
ax[1].set_yscale('log')
ax[1].grid(True, which='both')
ax[1].set_ylabel('$Q_e$')
ax[1].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[1].set_ylim(1e3, 1e6)
c = 'Qi_hp'
ax[2].boxplot(summary_table_new['Qi_hp'])
ax[2].set_ylabel('$Q_i$ (hp)')
ax[2].set_yscale('log')
ax[2].grid(True, which='both')
ax[2].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[2].set_ylim(1e3, 1e6)
c = 'Qi_lp'
ax[3].boxplot(summary_table_new['Qi_lp'])
ax[3].set_ylabel('$Q_i$ (lp)')
ax[3].set_yscale('log')
ax[3].grid(True, which='both')
ax[3].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[3].set_ylim(1e3, 1e6)
plt.tight_layout()
plt.suptitle(sample_name)
plt.savefig('{0}/{1}-boxplot.pdf'.format(save_path, sample_name))

In [182]:
print(summary_table_new['Qe'].describe())

count      24.000000
mean     2608.708883
std      1164.310022
min       580.837884
25%      1558.634010
50%      2789.896500
75%      3450.048432
max      4645.003045
Name: Qe, dtype: float64


In [229]:
rf_switch.get_ch1_switch()

0

# bullshit

## measurements as a function of current

In [28]:
dac = nndac('TCPIP0::10.20.61.12::1000::SOCKET')
#coil_device = awg_tek
coil_device = dac

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [92]:
    pna.set_nop(9001)
    pna.set_average(1)
    pna.set_averages(1)
    pna.set_xlim(5.6e9, 6.7e9)
    pna.set_bandwidth(1)
    pna._visainstrument.write("SENS:AVER:MODE POIN")
    resonator_freq_span = 5e6
    powers = np.linspace(-60, -20, 21)
    def set_power(power):
        bw = 100
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweeper.sweep(pna, 
                              (powers, set_power, 'Power'), 
                              sample_name=sample_name,
                              measurement_type='resonator_powerscan',
                              type_revision='0',
                              metadata={})
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type, delay=delay_estimate)
    default_measurement_save_path(save_path, mkdir=True)
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



{}
load_exdir: file open time:  0.0020155906677246094
load_exdir: file open time:  0.0019979476928710938
load_exdir: file open time:  0.0009946823120117188
load time:  0.04198789596557617
pre_trace_time:  0.001976490020751953
trace 0 time:  0.028011083602905273
trace 1 time:  0.05497264862060547
trace 2 time:  0.08695840835571289
trace 3 time:  0.11994075775146484


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\memmap.py:335: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



NameError: name 'delay_estimate' is not defined

In [30]:
if coil_device != dac:
    raise ValueError('awg_tek is the active coil control device. This cell is for nndac.')
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = -1
current_nop = 21
#current_nop = 51
nndac_channels = [22]#[0]#[1]#[18,20]#[18, 20, 16]#[21,1,7]
#awg_tek_channels = [1,2,3]
#currents=np.linspace(-current_max,current_max,current_nop)
#voltages=np.linspace(-voltage_max,voltage_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(3001)
pna.set_bandwidth(200)
#pna.set_power(qubits[qubit_id]['r']['P']+55) 
pna.set_power(-50) #-30

def sleep(x,channel):
    dac.set_voltage(x, channel=channel)
    #time.sleep(4)
for nndac_channel_id in nndac_channels:
    
        dac.set_voltage(0, channel=nndac_channel_id)
        coil = 'NNDAC-'+str(nndac_channel_id)
        
        
        pna.set_xlim(5.6e9, 6.7e9)
        result = sweeper.sweep(pna, 
                        (voltages, lambda x: sleep(x, channel=nndac_channel_id), 'Coil #{} voltage'.format(coil)), 
                        sample_name=sample_name, 
                        measurement_type='resonator_scan',
                        metadata={'nndac_channel_id':nndac_channel_id,'pna_power':pna.get_power()})
                               #filename='Fluxonium 1902-057-2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        for nndac_channel in range(24):
            dac.set_voltage(0, channel=nndac_channel)#0)


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



{}


KeyboardInterrupt: 

In [8]:
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
awg_tek.check_cached = True
coil_device = awg_tek

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional
C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [15]:

if coil_device != awg_tek:
    raise ValueError('awg_tek is not the active coil control device. This cell is for awg_tek.')
#coil = 'z1'
#current_max = 1.5e-3

voltage_max = 0.5
current_nop = 41
# nndac_channels = [6,8,10]#[18,20]#[18, 20, 16]#[21,1,7]
awg_tek_channels = [3, 4]
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
pna.set_nop(9001)
pna.set_bandwidth(200)
pna.set_power(-50)#qubits[qubit_id]['r']['P']+60)

pna.set_status(1)
for awg_tek_channel in awg_tek_channels:
        awg_tek.set_output(1, channel=awg_tek_channel)
        pna.set_xlim(5.6e9, 6.7e9)
        result = sweeper.sweep(pna, 
                        (voltages, lambda x: coil_device.set_offset(x, channel=awg_tek_channel), 'Coil #{} voltage'.format(awg_tek_channel)), 
                        sample_name=sample_name, 
                        measurement_type='resonator_scan_coil',
                        metadata={'awg_channel_id': awg_tek_channel, 'pna_power':pna.get_power()})
        awg_tek.set_output(0, channel=coil_device_channel)
        for awg_tek_channel2 in awg_tek_channels:
            awg_tek.set_offset(0, channel=awg_tek_channel2)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



{}


VisaIOError: VI_ERROR_RSRC_LOCKED (-1073807345): Specified type of lock cannot be obtained, or specified operation cannot be performed, because the resource is locked.

In [14]:
awg_tek.set_offset(0, channel=4)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



True

In [51]:
if coil_device != dac:
    raise ValueError('awg_tek is the active coil control device. This cell is for nndac.')
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = -4.0
current_nop = 81
#current_nop = 51
nndac_channels = [1]#[1]#[18,20]#[18, 20, 16]#[21,1,7]
#awg_tek_channels = [1,2,3]
#currents=np.linspace(-current_max,current_max,current_nop)
#voltages=np.linspace(-voltage_max,voltage_max,current_nop)
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
pna.set_nop(3001)
pna.set_bandwidth(200)
#pna.set_power(qubits[qubit_id]['r']['P']+55) 
pna.set_power(-50) #-30

def sleep(x,channel):
    coil_device.set_voltage(x, channel=channel)
    #time.sleep(4)
for nndac_channel_id in nndac_channels:
    
        coil_device.set_voltage(0, channel=nndac_channel_id)
        coil = 'NNDAC-'+str(nndac_channel_id)
        
        
        pna.set_xlim(5.6e9, 6.7e9)
        result = sweeper.sweep(pna, 
                        (voltages, lambda x: sleep(x, channel=nndac_channel_id), 'Coil #{} voltage'.format(coil)), 
                        sample_name=sample_name, 
                        measurement_type='resonator_scan_coil',
                        metadata={'nndac_channel_id':nndac_channel_id,'pna_power':pna.get_power()})
                               #filename='Fluxonium 1902-057-2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        for nndac_channel in nndac_channels:
            coil_device.set_voltage(0, channel=nndac_channel)#0)

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



{}


KeyboardInterrupt: 

In [ ]:
save_exdir.load_exdir(Data[894].filename)

In [ ]:
voltages = np.linspace(-0.45, 0.45, 46)
dac_channels = [1,3,5]
lo1.set_status(0)
pna.set_power(-50)
pna.set_xlim(5.6e9, 6.7e9)
pna.set_nop(9001)
for dac_channel in dac_channels:
    for resonator_id, resonator_freq in enumerate(resonator_freqs):
        pna.set_span(10e6)
        pna.set_centerfreq(resonator_freq)
        try:
            sweeper.sweep(pna, 
              (voltages, lambda x: dac.set_voltage(x, channel=dac_channel), 'Coil voltages', 'V'),
              sample_name=sample_name, 
              measurement_type='single-tone-spectroscopy',
              metadata={'probe_power':pna.get_power(),
                        'resonator_id':resonator_id,
                        'dac_channel':dac_channel,
                        'bw':pna.get_bandwidth()})
        except:
            raise
        finally:
            dac.set_voltage(0, channel=dac_channel)

# Уточняем частоты

In [43]:
resonator_freqs = [5700410000.0 ,5721000000.0 ,5764300000.0 ,5808600000.0 ,5832800000.0 ,
5870970000.0 ,5909800000.0 ,5948300000.0 ,5981700000.0 ,6045900000.0 ,6068200000.0 ,
6101200000.0 ,6166900000.0 ,6193600000.0 ,6225900000.0 ,6294800000.0, 6320500000.0 ,6338800000.0,
6388700000.0 ,6425400000.0 ,6460800000.0 ,6486600000.0 ,6523300000.0 ,6557400000.0, 6606500000.0 ,
6657800000.0]

In [96]:
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



In [36]:
# RF switch for making sure we know what sample we are measuring
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')
CHASSIS = 0
SLOT_AWG1 = 3
SLOT_AWG2 = 5


awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
awg_tek.check_cached = True

sa = Agilent_N9030A('pxa', address = 'TCPIP0::10.20.61.56::inst0::INSTR')


lo_ex = lo1
lo_ro = pna

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


In [37]:
awg1 = Keysight_M3202A_S('awg1', CHASSIS, SLOT_AWG1)
awg2 = Keysight_M3202A_S('awg2', CHASSIS, SLOT_AWG2)
awg1.get_clock = lambda: 1e9
awg2.get_clock = lambda: 1e9

In [38]:
adc_device = TSW14J56_evm()
adc_device.timeout = 10
adc = TSW14J56_evm_reducer(adc_device)
adc.output_raw = True
adc.last_cov = False
adc.avg_cov = False
adc.resultnumber = False

ADS54J40 already programmed


# mixer bypass

In [39]:
# когда спектроскопия не рабтает и всё плохо, тыкай сюды!
# вектнй анлизатр цепей - в режим свипа по частоте
pna.set_sweep_mode("LIN")
# смесители - открыть
#awg_tek = awg
#for channel in range(0,4):
#awg._visainstrument.write('AWGC:DOUT1 OFF')
#awg._visainstrument.write('AWGC:DOUT2 OFF')
#awg._visainstrument.write('AWGC:DOUT3 OFF')
#awg._visainstrument.write('AWGC:DOUT4 OFF')
awg2.trigger_source_types = [0,6,6,6]
awg2.trigger_source_channels = [0, 4000,4000,4000]
awg2.trigger_delays = [30+10,7+10,7+10,7+10]
awg2.trigger_behaviours = [0,4,4,4]
awg2.set_trigger_mode(2)
awg1.set_trigger_mode(2)
awg1.trigger_source_types = [6,6,6,6]
awg1.trigger_source_channels = [4000, 4000,4000,4000]
awg1.trigger_delays = [7+10,7+10,7+10,7+10] # wait 60 ns for sync with awg2
awg1.trigger_behaviours = [4,4,4,4]
awg1.stop()
awg2.stop()
for channel in range(0,4):
    awg2.set_amplitude(0*0.05, channel=channel)
    awg2.set_offset(0*0.7,channel=channel)
    awg2.set_output(1, channel=channel)
    awg2.set_waveform(waveform = [0]*99500, channel=channel)
for channel in range(0,4):
    awg1.set_amplitude(0.03, channel=channel)
    awg1.set_offset(0.7,channel=channel)
    awg1.set_output(1, channel=channel)
    awg1.set_waveform(waveform =[0]*99500, channel=channel)
awg2.set_marker(length=200, delay=0, channel=0, pxi_channels=1)
awg1.set_marker(length=200, delay=0, channel=0, pxi_channels=2)
awg2.run()
awg1.run()
#     

# Задаём постояннmtые смещения на каналы 1 и 3
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=1)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=2)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=3)
#awg_tek.set_waveform([0]*awg_tek.get_nop(), channel=4)

In [61]:
#voltages = np.linspace(-4, 4, 101)
pump_freq = np.linspace(4.5e9, 7e9, 1001)
lo1.set_power(-20)
lo1.set_status(1)
pna.set_power(-50)
pna.set_bandwidth(50)
dac_channels = [0, 2, 4]
for resonator_id, resonator_freq in enumerate(resonator_freqs[24]):
    for dac_channel in dac_channels:
        pna.set_nop(1)
        pna.set_centerfreq(resonator_freq+0.5e6)
        sweeper.sweep(pna, 
              (voltages, lambda x: dac.set_voltage(x, channel=dac_channel), 'Coil voltages', 'V'),
              #(pump_freq, lo1.set_frequency, 'Pump power', 'dBm'),
              measurement_type='Two-tone-1D',
              sample_name=sample_name, 
              metadata={'probe_power':pna.get_power(),
                        'pump_power':lo1.get_power(),
                        'resonator_id':resonator_id,
                        'fr':resonator_freq+0.5e6,
                        'dac_channel': dac_channel,
                        'bw':50})

TypeError: 'float' object is not iterable

In [69]:
voltages = np.linspace(-1.5, 1.5, 101)
pump_freq = np.linspace(3.6e9, 5.6e9, 1001)
lo1.set_power(10)
lo1.set_status(1)
pna.set_power(-50)
pna.set_bandwidth(50)
dac_channels = [1]#, 3, 5]
resonator_id=24
resonator_freq=resonator_freqs[24]
#for resonator_id, resonator_freq in enumerate(resonator_freqs):
for dac_channel in dac_channels:
        pna.set_nop(1)
        pna.set_centerfreq(resonator_freq)
        sweeper.sweep(pna, 
              (voltages, lambda x: dac.set_voltage(x, channel=dac_channel), 'Coil voltages', 'V'),
              (pump_freq, lo1.set_frequency, 'Excitation frequency', 'Hz'),
              measurement_type='Two-tone-1D',
              sample_name=sample_name, 
              metadata={'probe_power':pna.get_power(),
                        'pump_power':lo1.get_power(),
                        'resonator_id':resonator_id,
                        'fr':resonator_freq,
                        'dac_channel': dac_channel,
                        'bw':50})

C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



{}


VisaIOError: VI_ERROR_RSRC_LOCKED (-1073807345): Specified type of lock cannot be obtained, or specified operation cannot be performed, because the resource is locked.

In [19]:
res_freqs=[5678312500.0 ,
5711218750.0 ,
5742812500.0 ,
5789734375.0 ,
5823343750.0 ,
5854093750.0 ,
5897500000.0 ,
5970718750.0 ,
6020171875.0 ,
6054203125.0 ,
6089875000.0 ,
6138015625.0 ,
6165765625.0 ,
6217984375.0 ,
6266453125.0 ,
6302265625.0 ,
6345062500.0 ,
6433656250.0 ,
6476171875.0 ,
6480906250.0 ,
6536593750.0 ,
6572828125.0 ,
6622093750.0 ,
6769046875.0 ]

In [23]:
 freq_spans=[9163085.089041043,
  7797727.543691004,
  8842588.207303768,
  8992905.27173226,
  6592364.940741043,
  11081543.786163105,
  11768944.96982719,
  12774575.262131136,
  10213945.752158666,
  7410654.597261313,
  22464426.05673034,
  12782723.884518592,
  23509228.349958308,
  20589116.635043986,
  11310598.79357906,
  6268435.009158206,
  41733138.21854943,
  11674929.662133653,
  1827.6698985258377,
  44292009.278918095,
  7911476.639116993,
  25648621.83179333,
  72533222.37617868,
  8913743.644106554]

In [24]:
if coil_device != dac:
    raise ValueError('awg_tek is the active coil control device. This cell is for nndac.')
#coil = 'z1'
#current_max = 1.5e-3
voltage_max = -4.0
current_nop = 201
#current_nop = 51
nndac_channels = [13]#[0]#[1]#[18,20]#[18, 20, 16]#[21,1,7]
voltages=np.linspace(-voltage_max,voltage_max,current_nop)
#current_src1.set_status(1)
#pna.set_nop(9001)
#pna.set_bandwidth(100)
#pna.set_power(qubits[qubit_id]['r']['P']+55) 
pna.set_power(-35) #-30

def sleep(x,channel):
    dac.set_voltage(x, channel=channel)
    #time.sleep(4)
for nndac_channel_id in nndac_channels:
    for res_id, res_freq in enumerate(res_freqs):
        dac.set_voltage(0, channel=nndac_channel_id)
        coil = 'NNDAC-'+str(nndac_channel_id)
        freq_span=freq_spans[res_id]
        pna.set_xlim(res_freq-freq_span/2, res_freq+freq_span/2)

        pna.set_nop(201)
        pna.set_average(1)
        pna.set_averages(1)
        pna.set_bandwidth(100)
        result = sweeper.sweep(pna, 
                        (voltages, lambda x: sleep(x, channel=nndac_channel_id), 'Coil #{} voltage'.format(coil)), 
                        sample_name=sample_name, 
                        measurement_type='resonator_skan',
                        metadata={'res_freq':res_freq,'freq_span':freq_span,'res_id':res_id,'nndac_channel_id':nndac_channel_id,'pna_power':pna.get_power()})
                               #filename='Fluxonium 1902-057-2 Coil #{} anticrossings qubit {}'.format(coil, qubit_id))
        for nndac_channel in range(24):
            dac.set_voltage(0, channel=nndac_channel)#0)


C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:573: FutureWarning:

ask is deprecated and will be removed in 1.10, use query instead.



{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 11:23:09.472991, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[5673730957.46, 5682894042.54] (201 points) without setter
data: [[-5.67678404-3.61510229j -5.73352242-3.52968144j -5.79082298-3.43497109j
  ...  1.88219666+4.62499046j  1.96003091+4.59894848j
   2.03631783+4.57292128j]
 [-5.71409607-3.59566784j -5.78272915-3.49424815j -5.85087061-3.39435339j
  ...  1.87214029+4.61967802j  1.95630801+4.59899712j
   2.02725077+4.5741725j ]
 [-5.69213676-3.6031208j  -5.76047802-3.50824547j -5.83268404-3.41082788j
  ...  1.8736136 +4.62114429j  1.94817221+4.59469557j
   2.02573419+4.56921148j]
 ...
 [-5.67181873-3.62145519j -5.7372756 -3.5255599j  -5.80598545-3.42845368j
  ...  1.86624682+4.62136078j  1.94404817+4.59474611j
   2.02242303+4.56791878j]
 [-5.67

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\memmap.py:335: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.



{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 11:29:29.495578, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[5707319886.23, 5715117613.77] (201 points) without setter
data: [[ 3.47356606-3.11676669j  3.42812347-3.16357899j  3.3740685 -3.20461845j
  ... -6.44598103-2.1142652j  -6.47073793-2.00753999j
  -6.4868331 -1.90543902j]
 [ 3.47677898-3.12097859j  3.4251492 -3.16179705j  3.38009763-3.20502329j
  ... -6.44230747-2.11413097j -6.46152878-2.00806165j
  -6.48553562-1.90731478j]
 [ 3.47753978-3.11937571j  3.42603326-3.16422749j  3.37848735-3.20198226j
  ... -6.44456816-2.11633325j -6.46221828-2.01280904j
  -6.48114061-1.90873361j]
 ...
 [ 3.48561096-3.10636187j  3.43358874-3.14704418j  3.38282037-3.18648982j
  ... -6.43012476-2.13570428j -6.45335579-2.02474976j
  -6.47118759-1.92182183j]
 [ 3.48

load time:  0.038988351821899414
pre_trace_time:  0.0009751319885253906
trace 0 time:  0.025011062622070312
trace 1 time:  0.04699420928955078
trace 2 time:  0.06998467445373535
trace 3 time:  0.0939779281616211
{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 11:42:09.211891, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[5785237922.36, 5794230827.64] (201 points) without setter
data: [[ 4.22653961-3.49027562j  4.16796255-3.57292438j  4.10775137-3.67520022j
  ... -3.65688825-0.08413521j -3.66450477-0.0312712j
  -3.66888165+0.02956377j]
 [ 4.22298861-3.49345183j  4.17211342-3.57519221j  4.11489201-3.65185905j
  ... -3.65548086-0.08481635j -3.66180229-0.02670756j
  -3.67013025+0.03003416j]
 [ 4.22516918-3.49013114j  4.17368364-3.57167578j  4.11984158-3.65224004j
  ... -3.65373802-0.08695614j -3.6632

...

load_exdir: file open time:  0.0019903182983398438
load_exdir: file open time:  0.0010175704956054688
load_exdir: file open time:  0.0010166168212890625
load time:  0.03796553611755371
pre_trace_time:  0.0
trace 0 time:  0.02400660514831543
trace 1 time:  0.04599618911743164
trace 2 time:  0.06898760795593262
trace 3 time:  0.09097456932067871
{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 12:58:09.091835, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[6299131407.5, 6305399842.5] (201 points) without setter
data: [[-2.72609162-4.73320627j -2.78078246-4.69595766j -2.83445954-4.66189718j
  ... -5.10552931+3.85477471j -5.05969477+3.91639757j
  -5.0137763 +3.9741292j ]
 [-2.73974991-4.72134018j -2.78701425-4.68676805j -2.8446672 -4.65058136j
  ... -5.11159611+3.85242128j -5.05973911+3.91100907j
  -5.

load_exdir: file open time:  0.0019991397857666016
load_exdir: file open time:  0.0009999275207519531
load_exdir: file open time:  0.0009996891021728516
load time:  0.037981510162353516
pre_trace_time:  0.0010001659393310547
trace 0 time:  0.022988319396972656
trace 1 time:  0.04497790336608887
trace 2 time:  0.06696701049804688
trace 3 time:  0.08995628356933594
{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 13:10:49.345563, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[6427818785.17, 6439493714.83] (201 points) without setter
data: [[ 6.00021362+1.93664253j  6.04024029+1.81046569j  6.07370567+1.6801393j
  ... -5.8255949 -0.46884102j -5.83812475-0.34784272j
  -5.84655046-0.22721635j]
 [ 5.99933672+1.9339776j   6.04076576+1.80761051j  6.0739398 +1.67893898j
  ... -5.82645941-0.47052664j -5.83698

load_exdir: file open time:  0.0010044574737548828
load_exdir: file open time:  0.0009989738464355469
load_exdir: file open time:  0.0019974708557128906
load time:  0.03798484802246094
pre_trace_time:  0.0010020732879638672
trace 0 time:  0.023983478546142578
trace 1 time:  0.04597139358520508
trace 2 time:  0.06895971298217773
trace 3 time:  0.09094977378845215
{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 13:23:25.241202, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[6458760245.36, 6503052254.64] (201 points) without setter
data: [[-5.90341568+0.94428176j -5.81274366+1.37751377j -5.69362402+1.79696345j
  ... -1.7822051 +4.44101572j -1.46131682+4.57107449j
  -1.12924099+4.67887449j]
 [-5.90195942+0.94990289j -5.81439972+1.38186109j -5.6961894 +1.81001949j
  ... -1.77927947+4.44192314j -1.45813

load_exdir: file open time:  0.003016948699951172
load_exdir: file open time:  0.0010025501251220703
load_exdir: file open time:  0.0009951591491699219
load time:  0.04098343849182129
pre_trace_time:  0.0
trace 0 time:  0.025983333587646484
trace 1 time:  0.04897260665893555
trace 2 time:  0.07195925712585449
trace 3 time:  0.09695267677307129
{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 13:36:05.727461, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[6560003814.08, 6585652435.92] (201 points) without setter
data: [[-2.35352707+5.75398064j -2.09806061+5.8502574j  -1.8428477 +5.93469191j
  ...  5.76217222+2.02990556j  5.84779167+1.77941024j
   5.92229891+1.5231384j ]
 [-2.34847569+5.7497406j  -2.0971849 +5.8467617j  -1.83801699+5.93370533j
  ...  5.75931168+2.03969383j  5.84501028+1.78137398j
   

load_exdir: file open time:  0.0020003318786621094
load_exdir: file open time:  0.0019989013671875
load_exdir: file open time:  0.0009992122650146484
load time:  0.03898787498474121
pre_trace_time:  0.0009984970092773438
trace 0 time:  0.023983001708984375
trace 1 time:  0.0459744930267334
trace 2 time:  0.0679631233215332
trace 3 time:  0.09195208549499512
{}
<bound method database.update_in_database of <qsweepy.database.database object at 0x000002873948B7F0>> start: 2019-04-25 13:48:46.954687, started/done/total sweeps: 0/201/201, 
Measured data: 
'S-parameter': parameters: Coil #NNDAC-13 voltage (),:[-4.0, 4.0] (201 points) with setter
Frequency (Hz),:[6764590003.18, 6773503746.82] (201 points) without setter
data: [[ 2.24341106-5.30087805j  2.16250205-5.33351755j  2.07320166-5.36668396j
  ... -4.8989377 +2.68543148j -4.85309649+2.76074624j
  -4.80829334+2.84136796j]
 [ 2.20183992-5.3107233j   2.11000204-5.3449769j   2.02248073-5.37678337j
  ... -4.89978504+2.67699289j -4.8567071 +2

In [23]:
freqs=measurement.get_data()[1]
powers=measurement.get_data()[0]
max_power_S21 = 10**(measurement.get_data()[2][np.argmax(powers), :]/20)*np.exp(1j*measurement.get_data()[3][np.argmax(powers), :])

fitter = circuit.reflection_port(f_data=freqs, z_data_raw=max_power_S21)
print(fitter.do_calibration(freqs, max_power_S21))
fitter.autofit()
fitter.plotall()
print(fitter.fitresults)
print(fitter.get_single_photon_limit())

(8.7617753961513634e-08, 0.00021846153804807672, 0.78230385058357799, 9473514161.3102646, 987237.85108576983, 0.0, 9480474204.3878784)
{'theta0': -0.87036269078204709, 'Qi_err': 4279766.7041571699, 'Ql_err': 109.19720855604572, 'Qc_err': 91.415426058562019, 'fr': 9482648929.5436897, 'Qc': 763.26702083792111, 'Qi': 198307.89384138637, 'fr_err': 516643.97835309943, 'Ql': 760.34054699522324, 'chi_square': 1.0765021049802501}
-129.081045603


In [6]:
measurement = dm.load_measurement('C:/qtlab/ReiData/data/Feb 27 2017/16-17-54/Res Nb CEXP 17.01.2016 switch 4-2 SH A anti-korenkovian warm')

In [28]:
%matplotlib qt5

In [378]:
names=[r"C:\Data\2018-10-12\01-04-11-1808_072-2 Coil #NNDAC-6 anticrossings qubit 1\1808_072-2 Coil #NNDAC-6 anticrossings qubit 1.pkl",
  r'C:\Data\2018-10-12\01-07-54-1808_072-2 Coil #NNDAC-8 anticrossings qubit 2\1808_072-2 Coil #NNDAC-8 anticrossings qubit 2.pkl',
      r'C:\Data\2018-10-12\03-48-08-1808_072-2 Coil #NNDAC-10 anticrossings qubit 3\1808_072-2 Coil #NNDAC-10 anticrossings qubit 3.pkl',
     
       r'C:\Data\2018-10-12\03-40-43-1808_072-2 Coil #NNDAC-6 anticrossings qubit 1\1808_072-2 Coil #NNDAC-6 anticrossings qubit 1.pkl'
]

In [517]:
num=2

In [518]:
f = open(names[num],'rb')
data = pickle.load(f)

In [519]:
freq=data[1]['S-parameter'][1][1]
cur = data[1]['S-parameter'][1][0]
s21 = data[1]["S-parameter"][2]

In [523]:
plt.pcolormesh(cur,freq,np.imag(s21).T)
plt.show()

In [524]:
plt.plot(freq,np.imag(s21[20]))
plt.show()

In [525]:
fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(data[1]["S-parameter"], cur, resonator_type="reflection_port")

In [526]:
fitdata

,Qc,Qc_err,Qi,Qi_err,Ql,Ql_err,chi_square,fr,fr_err,power,single_photon_limit,theta0
0,9262.989964,142.612175,102958.281835,12528.012275,8498.402452,160.928894,0.005399,6.735200e+09,4398.452779,-4.00,-142.178465,0.033198
1,10979.526530,136.986154,140700.671175,20724.511326,10184.762252,165.779408,0.005556,6.735231e+09,3933.047054,-3.92,-143.012372,0.027002
2,10555.493403,94.280144,133316.851833,13634.316960,9781.067707,112.242598,0.002682,6.735207e+09,2709.110871,-3.84,-142.832162,0.024882
3,11099.003553,183.920711,137581.722830,23891.859914,10270.463884,218.612187,0.007928,6.735204e+09,4666.945205,-3.76,-143.038188,0.024866
4,10130.893108,62.119434,120850.835909,8581.236787,9347.310573,77.615613,0.001445,6.735167e+09,2002.704481,-3.68,-142.616530,0.029770
5,10341.844877,67.772927,127413.888461,10695.147427,9565.443395,87.147707,0.001715,6.735157e+09,2054.196025,-3.60,-142.727409,0.028327
6,9891.585134,72.733032,112473.472145,8947.314550,9091.982219,94.764240,0.001933,6.735128e+09,2366.272581,-3.52,-142.479836,0.034560
7,9723.624677,88.090088,116117.435737,10183.398575,8972.289012,105.070459,0.002413,6.735126e+09,2758.064844,-3.44,-142.439110,0.034627
8,13723.385582,768.631217,185124.674081,134905.987590,12776.273942,965.008794,0.087975,6.735241e+09,12983.616509,-3.36,-144.012649,0.007494
9,10947.898680,150.763842,135517.431960,21055.132412,10129.572015,180.864744,0.006108,6.735169e+09,4013.971397,-3.28,-142.977785,0.026964


In [527]:
fr=fitdata['fr']
Q=fitdata['Qi']

In [528]:
spl=fitdata['single_photon_limit']

In [529]:
spl.mean()

-142.81925254053328

In [532]:
plt.plot(cur,Q)
plt.show()

In [531]:
plt.hist(Q)

(array([ 2.,  1.,  3.,  4., 19., 26., 30., 10.,  5.,  1.]),
 array([ 74677.85003448,  85722.53243915,  96767.21484383, 107811.89724851,
        118856.57965319, 129901.26205787, 140945.94446254, 151990.62686722,
        163035.3092719 , 174079.99167658, 185124.67408125]),
 <a list of 10 Patch objects>)

In [533]:
trh={'Qmin':[70e3,70e3,100e3],
    'Qmax':[110e3,6e5,160e3]}

In [537]:
cur_new = cur

In [432]:
trh['Qmax'][1]

150000.0

In [535]:
#indexfr= fr[(fr<6e9) | (fr>7e9)].index
#indexQ= Q[(Q<1e4) | (Q>0.75e6)].index
indexfr= fr[(fr<6.7e9) | (fr>6.8e9)].index
indexQ= Q[(Q<trh['Qmin'][2]) | (Q>trh['Qmax'][2])].index

In [538]:
indexQ

Int64Index([8, 19, 37, 62, 64, 66, 71, 74, 75, 79, 80, 82], dtype='int64')

In [304]:
indexfr

Int64Index([], dtype='int64')

In [539]:
cur_new = np.delete(cur_new,indexQ)
cur_new.shape

(89,)

In [402]:
cur_new = np.delete(cur_new,indexfr)
cur_new.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  """Entry point for launching an IPython kernel.


(0,)

In [410]:
np.mean(Q)

91750.59407702721

In [543]:
np.mean(fitdata['Ql'])

9786.511281884079

In [297]:
fitdata=fitdata.drop(indexfr)

In [540]:
fitdata=fitdata.drop(indexQ)

In [312]:
plt.figure(2)
plt.plot(cur_new,fitdata["fr"])
plt.show()

In [247]:
fr.shape

(87,)

In [315]:
plt.plot(cur_new,fr)
plt.show()

In [541]:
#plt.figure()
plt.scatter(cur_new,fitdata['Qi'])
plt.xlabel("Напряжение,V")
plt.ylabel("Q_i")
plt.show()

In [509]:
np.mean(2*Q/(2*np.pi*fr))

9.00153224502442

In [329]:
np.mean(2*fitdata['Ql']/(2*np.pi*fr))

4.6240390369067573e-07